In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard

from keras.models import Model
from keras import regularizers

from keras.backend import eval

import pandas as pd
import numpy as np

Using TensorFlow backend.


In [21]:
df = pd.read_csv('data/ETHBTC_1517303160000_1547498100000.csv')
df.head(10)

,OPEN_TIME,OPEN,HIGH,LOW,CLOSE,VOLUME,CLOSE_TIME,QUOTE_VOL,NUM_TRADES,TAKE_BASE_VOL,TAKE_QUOTE_VOL,EXTRA
0,1517303100000,10629400,10673700,10629300,10640100,17785000000,1517303159999,1892484646,412,10429400000,1110434538,0
1,1517303160000,10640100,10643300,10619800,10630100,15433400000,1517303219999,1640768918,276,4288000000,456085864,0
2,1517303220000,10621100,10640000,10621000,10625000,9354600000,1517303279999,994314138,255,4298600000,457129449,0
3,1517303280000,10626200,10634500,10622600,10630800,6746400000,1517303339999,716919384,184,3005400000,319476265,0
4,1517303340000,10630800,10639000,10622600,10632600,7650900000,1517303399999,813139316,182,3471000000,369112330,0
5,1517303400000,10632600,10632600,10620000,10622400,6602300000,1517303459999,701424997,191,2897600000,307888440,0
6,1517303460000,10624100,10655300,10621900,10633100,18857000000,1517303519999,2005967577,415,8383700000,892185162,0
7,1517303520000,10633100,10648400,10628900,10632200,10156400000,1517303579999,1080391056,270,6018500000,640444989,0
8,1517303580000,10632600,10649200,10632200,10644200,15437200000,1517303639999,1642054242,282,4604000000,490073186,0
9,1517303640000,10644000,10690000,10633500,10644300,19295500000,1517303699999,2054704543,352,11366500000,1211303174,0


In [22]:
data = df.loc[:, ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME']].values
data.shape
data = data/data.max(axis=0).astype(float)
data[:5]

(500000, 5)

array([[0.85942755, 0.86217982, 0.86067206, 0.86029269, 0.00995129],
       [0.86029269, 0.85972423, 0.85990283, 0.85948415, 0.00863549],
       [0.85875647, 0.85945767, 0.86      , 0.8590718 , 0.0052342 ],
       [0.85916882, 0.8590134 , 0.86012955, 0.85954075, 0.00377483],
       [0.85954075, 0.85937689, 0.86012955, 0.85968629, 0.00428093]])

In [23]:
inputs = Input(shape=(100, 5))
encoded = LSTM(150)(inputs)

decoded = RepeatVector(100)(encoded)
decoded = LSTM(150, return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(5))(decoded)

sequence_autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)

sequence_autoencoder.compile(optimizer='adam', loss='mse', metrics=['mae'])
sequence_autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 100, 5)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 150)               93600     
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 100, 150)          0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 100, 150)          180600    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 100, 5)            755       
Total params: 274,955
Trainable params: 274,955
Non-trainable params: 0
_________________________________________________________________


In [24]:
X = data.reshape(-1, 100, 5)
X.shape

(5000, 100, 5)

In [25]:
eval(sequence_autoencoder.optimizer.lr)

0.001

In [26]:
early_stop  = EarlyStopping(monitor='val_loss', patience=20, verbose=1)
reduce_lr   = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.000001, verbose=1)
tensorboard = TensorBoard(log_dir='/Users/abhi/Desktop/crypter/logs')

sequence_autoencoder.fit(X[:4000], X[:4000],
                        epochs=100,
                        batch_size=256,
                        shuffle=True,
                        callbacks=[early_stop, reduce_lr, tensorboard],
                        validation_data=(X[4000:], X[4000:]))

Train on 4000 samples, validate on 1000 samples
Epoch 1/100
4000/4000 [==============================] - 22s 6ms/step - loss: 0.0355 - mean_absolute_error: 0.1238 - val_loss: 0.0042 - val_mean_absolute_error: 0.0576
Epoch 2/100
4000/4000 [==============================] - 31s 8ms/step - loss: 0.0047 - mean_absolute_error: 0.0458 - val_loss: 0.0025 - val_mean_absolute_error: 0.0446
Epoch 3/100
4000/4000 [==============================] - 24s 6ms/step - loss: 0.0027 - mean_absolute_error: 0.0284 - val_loss: 0.0013 - val_mean_absolute_error: 0.0313
Epoch 4/100
4000/4000 [==============================] - 24s 6ms/step - loss: 0.0018 - mean_absolute_error: 0.0192 - val_loss: 6.4673e-04 - val_mean_absolute_error: 0.0195
Epoch 5/100
4000/4000 [==============================] - 25s 6ms/step - loss: 0.0014 - mean_absolute_error: 0.0125 - val_loss: 2.9549e-04 - val_mean_absolute_error: 0.0064
Epoch 6/100
4000/4000 [==============================] - 23s 6ms/step - loss: 0.0011 - mean_absolute_err

Epoch 46/100
4000/4000 [==============================] - 20s 5ms/step - loss: 2.5811e-05 - mean_absolute_error: 0.0026 - val_loss: 6.0314e-05 - val_mean_absolute_error: 0.0031
Epoch 47/100
4000/4000 [==============================] - 22s 6ms/step - loss: 2.5637e-05 - mean_absolute_error: 0.0025 - val_loss: 5.9873e-05 - val_mean_absolute_error: 0.0031
Epoch 48/100
4000/4000 [==============================] - 22s 5ms/step - loss: 2.5569e-05 - mean_absolute_error: 0.0026 - val_loss: 5.9807e-05 - val_mean_absolute_error: 0.0031
Epoch 49/100
4000/4000 [==============================] - 24s 6ms/step - loss: 2.5385e-05 - mean_absolute_error: 0.0025 - val_loss: 5.9549e-05 - val_mean_absolute_error: 0.0031
Epoch 50/100
4000/4000 [==============================] - 23s 6ms/step - loss: 2.5225e-05 - mean_absolute_error: 0.0025 - val_loss: 5.9510e-05 - val_mean_absolute_error: 0.0031
Epoch 51/100
4000/4000 [==============================] - 21s 5ms/step - loss: 2.5112e-05 - mean_absolute_error: 0.

4000/4000 [==============================] - 20s 5ms/step - loss: 2.3661e-05 - mean_absolute_error: 0.0024 - val_loss: 5.7515e-05 - val_mean_absolute_error: 0.0030
Epoch 91/100
4000/4000 [==============================] - 20s 5ms/step - loss: 2.3655e-05 - mean_absolute_error: 0.0024 - val_loss: 5.7482e-05 - val_mean_absolute_error: 0.0030
Epoch 92/100
4000/4000 [==============================] - 20s 5ms/step - loss: 2.3644e-05 - mean_absolute_error: 0.0024 - val_loss: 5.7438e-05 - val_mean_absolute_error: 0.0030

Epoch 00092: ReduceLROnPlateau reducing learning rate to 3.90625018554e-06.
Epoch 93/100
4000/4000 [==============================] - 20s 5ms/step - loss: 2.3637e-05 - mean_absolute_error: 0.0025 - val_loss: 5.7545e-05 - val_mean_absolute_error: 0.0030
Epoch 94/100
4000/4000 [==============================] - 20s 5ms/step - loss: 2.3627e-05 - mean_absolute_error: 0.0024 - val_loss: 5.7458e-05 - val_mean_absolute_error: 0.0030
Epoch 95/100
4000/4000 [===========================

In [ ]:
X[4000:].std(axis=1).mean(axis=0)

In [17]:
y_true = X[4000:]
y_pred = sequence_autoencoder.predict(X[4000:])

In [18]:
np.abs(y_true - y_pred).mean(axis=1)[:5]
np.abs(y_true - y_pred).mean(axis=1).mean(axis=0)
np.abs(y_true - y_pred).mean()
np.abs(y_true - y_pred).std(axis=1)[:5]
np.abs(y_true - y_pred).std(axis=1).mean(axis=0)

array([[0.00143693, 0.00159314, 0.0013779 , 0.00127187, 0.00468014],
       [0.00128457, 0.00148816, 0.00162846, 0.00149902, 0.00440173],
       [0.00142833, 0.0014421 , 0.00141383, 0.0013673 , 0.00568453],
       [0.00210694, 0.00216468, 0.00236947, 0.00232669, 0.00652406],
       [0.00231557, 0.00179632, 0.00197361, 0.00203877, 0.00636409]])

array([0.0022856 , 0.00219143, 0.00238483, 0.00240553, 0.0065297 ])

0.0031594172555708443

array([[0.00468096, 0.00417929, 0.00479477, 0.00471278, 0.0015732 ],
       [0.00480767, 0.0042691 , 0.00478902, 0.00474618, 0.00278236],
       [0.00482573, 0.00433056, 0.00487672, 0.00483059, 0.01132021],
       [0.00484436, 0.00434562, 0.00483098, 0.00481142, 0.01538759],
       [0.00456308, 0.00414669, 0.00475901, 0.00460063, 0.00852969]])

array([0.00463314, 0.00412722, 0.00464213, 0.00455984, 0.00776814])

In [20]:
X_encoded = encoder.predict(X)